介绍有关梯度，GPU,张量连接，内存的知识

如果 out=f(x)是标量，out.backward() 会计算 ∂out/∂x  
当out是矩阵时，out.backward() 默认不允许直接调用（因为梯度是矩阵，方向不明确）  
必须提供一个与 out 形状相同的张量 gradient（即 grad_outputs），表示 out 的每个元素对最终损失的贡献权重。  
torch.ones_like(out) 对 out 的每个元素求和（即 out.sum()）  
对求和后的标量结果调用 backward()

In [92]:
import torch
import numpy as np
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = torch.tensor([[5., 6.], [7., 8.]], requires_grad=True)#定义 x ，y并启用梯度跟踪
out = (x.pow(2)*y).sum()#对x的每个元素平方，再对所有元素求和
out.backward()#out最好是标量，不然很复杂
print(x.grad)#表示OUT对x的梯度
print(y.grad)

tensor([[10., 24.],
        [42., 64.]])
tensor([[ 1.,  4.],
        [ 9., 16.]])


数据放到GPU或者CPU上进行计算

In [93]:
x = torch.tensor([1, 2])#默认是放在cpu  
if torch.cuda.is_available():         
    device = torch.device('cuda')     
    y = torch.ones_like(x, device=device)
    x = x.to(device)                  
    z = x + y                         
    print(z)                          
    print(z.to('cpu'), torch.double)  

tensor([2, 3], device='cuda:0')
tensor([2, 3]) torch.float64


张量连结（concatenate）提供张量列表，给出沿哪个轴连结

In [94]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

节省内存  
Python首先计算Y + X，为结果分配新的内存，然后使Y指向内存中的这个新位置

In [95]:
before = id(Y)
Y = Y + X
id(Y) == before

False

执行原地更新，即Y的地址保持不变，只是更新上面的值  
1.切片表示法Y[:] = <expression>
2.+=操作

In [96]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))


id(Z): 125444138381728
id(Z): 125444138381728


In [97]:
before = id(Y)
Y += X
id(Y) == before

True

下面详细剖析计算图的知识  
<pre>
计算图是其自动微分功能的核心，它本质上是一个​​有向无环图（DAG）​​，用于动态记录所有的张量操作，以便在反向传播时自动计算梯度
其最大的特点是​​动态性​​，计算图是在代码运行时即时构建的，而非预先静态定义
主要由以下两个基本元素构成：
    ​节点 (Node)​​：代表​​数据​，​即​​张量（Tensor），节点分为叶子节点与非叶子节点
        叶子节点：最底端的根
            用户直接创建：如torch.tensor(..., requires_grad=True)。
            无父节点：不依赖其他张量生成。
            梯度存储：梯度存储在leaf_tensor.grad中。
            使用is_leaf属性判断
        非叶子节点：由叶子节点计算得到的中间节点(中间​张量数据)
            梯度默认在反向传播后释放（内存优化）。
            可通过retain_grad()强制保留。

    边 (Edge)​​：代表​​操作（Operation）​​，即对张量进行的数学运算（如加法、乘法），连接着各个节点，方向表示数据流​​
        连接了参与运算的输入张量（节点）和输出张量（节点）
        边的方向指示了计算和梯度传播的方向。前向传播时，数据沿边指向的方向计算；反向传播时，梯度则沿边的反方向传播

重要注意事项
    ​​梯度累积与清零​​：由于梯度会累积在叶节点的 .grad中，在训练循环的每一步​​反向传播之前，需要手动将优化器的梯度清零​​（optimizer.zero_grad()），否则梯度会不断累加，导致训练不稳定
    ​​禁用梯度跟踪​​：在进行模型推理或计算不需要梯度的中间值时，可以使用 with torch.no_grad():上下文管理器。这会暂时禁用计算图的构建，显著节省内存并加速计算
    ​​分离张量​​：使用 .detach()方法可以从当前计算图中分离出一个张量。新张量与原始张量数据相同，但不再参与梯度计算，其 requires_grad为 False，通过设置为true会使其成为新的叶节点。
    ​​原位操作（In-place）的限制​​：应避免对 requires_grad=True的​​叶节点​​进行原位操作（如 x.data += 1）。这会直接修改原始张量的值，可能会破坏计算图并导致梯度计算错误


递归遍历打印计算图

In [98]:
from torchviz import make_dot
# 创建测试计算图
x = torch.tensor([2.0], requires_grad=True)
y = torch.tensor([3.0], requires_grad=True)
z = x * y                  # MulBackward0
w = x + y                  # AddBackward0
out = z + w * 2            # AddBackward0 (包含两个输入)
# 安全可视化计算图
dot = make_dot(
    out,  # 输出张量
    params={#只能输入叶子张量进行重命名
        "叶子x": x,
        "叶子y": y
    },
    show_attrs=True
)
dot.render("2_model_visualization", format="svg")

'2_model_visualization.svg'

backward()反向传播的必要条件：
    存在从当前张量到叶子节点的完整计算图路径，  
    而计算图的动态构建需满足张量参与运算且requires_grad=True，  
    叶子节点则是由用户直接创建且requires_grad=True的张量  

计算示例  
演示了叶子节点、中间节点、中间节点梯度保留、梯度分离、新建叶子节点等操作

In [99]:
# 建立叶子节点
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([3.0])
y.requires_grad=True

#创建中间节点
ind_a = x * y*2
ind_b = x + y

#保留一个中间节点的梯度
ind_a.retain_grad()

# 使用detach新建新分支
det_a=ind_b.detach()
print(det_a.requires_grad)#false
# 分离出来的张量转换为叶子节点
det_a.requires_grad_(True)

#计算最终结果
z=ind_a*ind_b+ind_b
c=det_a*y

False


进行backward计算结果节点对于叶子节点的梯度

In [100]:
#反向传播
z.backward()  

#验证所有节点的required_grad
print(x.requires_grad,y.requires_grad,ind_a.requires_grad,ind_b.requires_grad,z.requires_grad)

#验证是否是叶子节点
print(x.is_leaf,y.is_leaf,ind_a.is_leaf,ind_b.is_leaf,z.is_leaf)

#打印梯度
print(x.grad,y.grad)  
print(ind_a.grad,ind_b.grad)  # 非叶子节点默认释放，输出 None（除非使用 retain_grad）

#梯度清零，避免累加
x.grad.zero_()
y.grad.zero_()
ind_a.grad.zero_()
#反向传播
c.backward()
print(det_a.grad,y.grad)
print(det_a.is_leaf)

True True True True True
True True False False False
tensor([31.]) tensor([15.])
tensor([4.]) None
tensor([3.]) tensor([4.])
True


/tmp/ipykernel_41050/3445178771.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647352509/work/build/aten/src/ATen/core/TensorBody.h:489.)
  print(ind_a.grad,ind_b.grad)  # 非叶子节点默认释放，输出 None（除非使用 retain_grad）


如果要再次计算梯度，首先进行梯度清零，然后重新通过计算过程

In [101]:
#梯度清零，避免累加
det_a.grad.zero_()
x.grad.zero_()
y.grad.zero_()
ind_a.grad.zero_()

#再次经过计算过程
ind_a = x * y*2
ind_a.retain_grad()
ind_b = x + y
z=ind_a*ind_b+ind_b
#再次backward
z.backward()  
#打印梯度
print(x.grad,y.grad)  
print(ind_a.grad,ind_b.grad)  # 非叶子节点默认释放，输出 None（除非使用 retain_grad）

#梯度清零，避免累加
det_a.grad.zero_()
x.grad.zero_()
y.grad.zero_()
ind_a.grad.zero_()

c=det_a*y
#反向传播
c.backward()
print(det_a.grad,y.grad)
print(det_a.is_leaf)

tensor([31.]) tensor([15.])
tensor([4.]) None
tensor([3.]) tensor([4.])
True


/tmp/ipykernel_41050/1648224921.py:16: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647352509/work/build/aten/src/ATen/core/TensorBody.h:489.)
  print(ind_a.grad,ind_b.grad)  # 非叶子节点默认释放，输出 None（除非使用 retain_grad）


with torch.no_grad()  与y=x.detach()的异同：  
<pre>
with torch.no_grad()  
    是一个上下文管理器，用于临时禁用整个代码块内的梯度计算。在该块内，所有涉及张量的操作（如前向传播、数学运算）都不会构建计算图，也不会记录梯度。  
    本质：通过全局设置 torch.set_grad_enabled(False) 阻断计算图的继续生成，节省内存和计算资源。
y=x.detach()
    是一个张量方法，用于从当前计算图中分离出一个张量副本。返回的新张量y与原张量共享数据（内存地址相同），但切断梯度传播路径，即反向传播时不会通过该张量回溯到原始计算图。
    修改原张量的数值会影响 detach 后的张量（反之亦然）  
    分离后的张量 requires_grad 属性为 false，分离后导致计算图断裂：
        如果需要跟踪返回的变量，需要显示设定required_grad为true使之成为叶子节点，再进行后续计算的跟踪和新计算图构建